In [ ]:
import polars as pl
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2
df = pl.read_parquet("pile.parquet")

In [ ]:
from synthetic_data.tasks.writing import WritingScoreAnnotate
from synthetic_data.utils import ldictl

input_df: pl.DataFrame = pl.read_parquet("../dataset_files/gutenberg_backtranslate.parquet")
task = WritingScoreAnnotate()

mock_batch = ldictl(input_df.head(4).to_dicts())
len(task.format_input_conversation(mock_batch))

In [8]:
from synthetic_data.generation import get_generation_wrapper


generation_wrapper = get_generation_wrapper("gpt-4o-mini")
completions = await generation_wrapper.generate(
    task.format_input_conversation(mock_batch)
)

In [9]:
output_rows_batch = task.format_output_rows(completions)

In [ ]:
print(output_rows_batch[0].keys())

In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from synthetic_data.tasks.writing import find_valid_chunks
from IPython.display import display, Markdown

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

df_sampled = df.sample(100)
for row in df.iter_rows(named=True):
    text = row["text"]
    paragraphs = super_cleaner(text)

    # find chunks of consecutive paragraphs with dialogue and at least 3 sentences
    valid_chunks = find_valid_chunks(paragraphs, tiktoken_encoder)
    display(Markdown(f"### {row['title']}"))
    for chunk in valid_chunks:
        display(Markdown(chunk[0]))

